In [1]:
import numpy as np
import pandas as pd
import os
import re

In [2]:
#2019 Data
url = 'https://www2.census.gov/programs-surveys/susb/tables/2019/county_3digitnaics_2019.xlsx'
susb = pd.read_excel(url,skiprows=2)
susb

,State,State Name,County,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Employment Noise Flag,"Annual Payroll\n($1,000)",Annual Payroll Noise Flag
0,1,Alabama,1,Autauga,--,Total,1: Total,797,867,11510,G,385755,G
1,1,Alabama,1,Autauga,--,Total,2: <20 employees,517,520,2272,G,68970,G
2,1,Alabama,1,Autauga,--,Total,3: 20-99 employees,80,86,2190,G,64449,G
3,1,Alabama,1,Autauga,--,Total,4: 100-499 employees,36,49,1395,H,38282,G
4,1,Alabama,1,Autauga,--,Total,5: 500+ employees,164,212,5653,G,214054,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500554,56,Wyoming,999,Statewide Wyoming,62,Health Care and Social Assistance,5: 500+ employees,3,3,15,J,919,G
500555,56,Wyoming,999,Statewide Wyoming,621,Ambulatory Health Care Services,1: Total,3,3,15,J,919,G
500556,56,Wyoming,999,Statewide Wyoming,621,Ambulatory Health Care Services,5: 500+ employees,3,3,15,J,919,G
500557,56,Wyoming,999,Statewide Wyoming,81,Other Services (except Public Administration),1: Total,4,4,14,J,1806,H


In [3]:
susb.columns = susb.columns.str.replace(' ','_')

In [4]:
## TODO: reduce dimensionality by reducing similar NAICS descriptions. 

# replacemap = {}

In [5]:
susb.query('State_Name=="Virginia"& Enterprise_Size=="1: Total"').groupby(['County_Name','NAICS_Description','Employment']).size()

County_Name  NAICS_Description                                                         Employment
Accomack     Accommodation                                                             259           1
             Accommodation and Food Services                                           1010          1
             Administrative and Support Services                                       398           1
             Administrative and Support and Waste Management and Remediation Services  421           1
             Agriculture, Forestry, Fishing and Hunting                                20            1
                                                                                                    ..
York         Truck Transportation                                                      40            1
             Utilities                                                                 61            2
             Waste Management and Remediation Services                        

In [6]:
susb_va = pd.DataFrame(susb.query('State_Name=="Virginia"& Enterprise_Size=="1: Total"')\
                           .groupby(['County_Name','NAICS_Description']).agg({'Employment':'sum'})).unstack()
susb_va.columns = susb_va.columns.map('_'.join).str.replace(' ', '_')
susb_va = susb_va.reset_index()

In [7]:
counties = {'Accomack':'Accomack County',
            'Albemarle':'Albemarle County',
            'Alleghany':'Alleghany County',
            'Amelia':'Amelia County',
            'Amherst':'Amherst County',
            'Appomattox':'Appomattox County',
            'Arlington':'Arlington County',
            'Augusta':'Augusta County',
            'Bath':'Bath County',
            'Bedford':'Bedford County',
            'Bland':'Bland County',
            'Botetourt':'Botetourt County',
            'Brunswick':'Brunswick County',
            'Buchanan':'Buchanan County',
            'Buckingham':'Buckingham County',
            'Campbell':'Campbell County',
            'Caroline':'Caroline County',
            'Carroll':'Carroll County',
            'Charles City':'Charles City County',
            'Charlotte':'Charlotte County',
            'Chesterfield':'Chesterfield County',
            'Clarke':'Clarke County',
            'Craig':'Craig County',
            'Culpeper':'Culpeper County',
            'Cumberland':'Cumberland County',
            'Dickenson':'Dickenson County',
            'Dinwiddie':'Dinwiddie County',
            'Essex':'Essex County',
            'Fairfax':'Fairfax County',
            'Fauquier':'Fauquier County',
            'Floyd':'Floyd County',
            'Fluvanna':'Fluvanna County',
            'Franklin':'Franklin County',
            'Frederick':'Frederick County',
            'Giles':'Giles County',
            'Gloucester':'Gloucester County',
            'Goochland':'Goochland County',
            'Grayson':'Grayson County',
            'Greene':'Greene County',
            'Greensville':'Greensville County',
            'Halifax':'Halifax County',
            'Hanover':'Hanover County',
            'Henrico':'Henrico County',
            'Henry':'Henry County',
            'Highland':'Highland County',
            'Isle of Wight':'Isle of Wight County',
            'James City':'James City County',
            'King George':'King George County',
            'King William':'King William County',
            'King and Queen':'King and Queen County',
            'Lancaster':'Lancaster County',
            'Lee':'Lee County',
            'Loudoun':'Loudoun County',
            'Louisa':'Louisa County',
            'Lunenburg':'Lunenburg County',
            'Madison':'Madison County',
            'Mathews':'Mathews County',
            'Mecklenburg':'Mecklenburg County',
            'Middlesex':'Middlesex County',
            'Montgomery':'Montgomery County',
            'Nelson':'Nelson County',
            'New Kent':'New Kent County',
            'Northampton':'Northampton County',
            'Northumberland':'Northumberland County',
            'Nottoway':'Nottoway County',
            'Orange':'Orange County',
            'Page':'Page County',
            'Patrick':'Patrick County',
            'Pittsylvania':'Pittsylvania County',
            'Powhatan':'Powhatan County',
            'Prince Edward':'Prince Edward County',
            'Prince George':'Prince George County',
            'Prince William':'Prince William County',
            'Pulaski':'Pulaski County',
            'Rappahannock':'Rappahannock County',
            'Richmond':'Richmond County',
            'Roanoke':'Roanoke County',
            'Rockbridge':'Rockbridge County',
            'Rockingham':'Rockingham County',
            'Russell':'Russell County',
            'Scott':'Scott County',
            'Shenandoah':'Shenandoah County',
            'Smyth':'Smyth County',
            'Southampton':'Southampton County',
            'Spotsylvania':'Spotsylvania County',
            'Stafford':'Stafford County',
            'Surry':'Surry County',
            'Sussex':'Sussex County',
            'Tazewell':'Tazewell County',
            'Warren':'Warren County',
            'Washington':'Washington County',
            'Westmoreland':'Westmoreland County',
            'Wise':'Wise County',
            'Wythe':'Wythe County',
            'York':'York County'}
susb_va.County_Name = susb_va.County_Name.replace(counties)

In [8]:
susb_va

,County_Name,Employment_Accommodation,Employment_Accommodation_and_Food_Services,Employment_Administrative_and_Support_Services,Employment_Administrative_and_Support_and_Waste_Management_and_Remediation_Services,"Employment_Agriculture,_Forestry,_Fishing_and_Hunting",Employment_Air_Transportation,Employment_Ambulatory_Health_Care_Services,"Employment_Amusement,_Gambling,_and_Recreation_Industries",Employment_Apparel_Manufacturing,...,Employment_Transportation_Equipment_Manufacturing,Employment_Transportation_and_Warehousing,Employment_Truck_Transportation,Employment_Utilities,Employment_Warehousing_and_Storage,Employment_Waste_Management_and_Remediation_Services,Employment_Water_Transportation,Employment_Wholesale_Electronic_Markets_and_Agents_and_Brokers,Employment_Wholesale_Trade,Employment_Wood_Product_Manufacturing
0,Accomack County,259.0,1010.0,398.0,421.0,20.0,NaN,694.0,73.0,NaN,...,NaN,114.0,57.0,168.0,NaN,NaN,NaN,NaN,130.0,NaN
1,Albemarle County,679.0,4921.0,2298.0,2374.0,109.0,138.0,4396.0,897.0,NaN,...,NaN,682.0,65.0,NaN,9.0,76.0,NaN,24.0,447.0,154.0
2,Alexandria city,1393.0,8667.0,13555.0,13639.0,NaN,113.0,2302.0,1023.0,NaN,...,NaN,1261.0,80.0,NaN,NaN,84.0,NaN,6.0,1277.0,NaN
3,Alleghany County,77.0,400.0,72.0,72.0,NaN,NaN,236.0,13.0,NaN,...,NaN,18.0,15.0,NaN,NaN,NaN,NaN,NaN,27.0,96.0
4,Amelia County,NaN,120.0,61.0,95.0,39.0,NaN,76.0,NaN,NaN,...,NaN,104.0,41.0,NaN,NaN,NaN,NaN,3.0,117.0,117.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Williamsburg city,1136.0,4025.0,360.0,366.0,NaN,NaN,304.0,148.0,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,171.0,NaN
130,Winchester city,212.0,2969.0,1444.0,1462.0,NaN,NaN,3273.0,227.0,NaN,...,NaN,182.0,130.0,NaN,43.0,NaN,NaN,15.0,520.0,NaN
131,Wise County,52.0,999.0,921.0,929.0,11.0,NaN,778.0,122.0,NaN,...,NaN,194.0,191.0,264.0,NaN,8.0,NaN,NaN,273.0,161.0
132,Wythe County,349.0,1495.0,361.0,399.0,NaN,NaN,609.0,29.0,NaN,...,689.0,504.0,437.0,NaN,NaN,38.0,NaN,NaN,465.0,33.0


In [9]:
## Not needed anymore
# pop_url = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv'
# pop = pd.read_csv(pop_url, encoding='latin1')
# pop_va = pop.query('STNAME=="Virginia"').loc[:,['CTYNAME','POPESTIMATE2019']]\
#                                         .rename({'CTYNAME':'County_Name','POPESTIMATE2019':'Population'}, axis = 1)
# pop_va

In [10]:
fed = pd.read_csv('va_federal_funding.csv')
fed = fed.drop('Unnamed: 0',axis=1)

In [11]:
fed['Org Name'] = fed['Org Name'].str.split(',').str[0]
fed = fed.rename({'Org Name':'County_Name'},axis=1)

In [12]:
#fed only has cities no counties
fed2019 = fed.query('Year==2019')
fed2019

,Year,State,CoC Name,County_Name,Program Name,Program Type,Award Amount
129,2019,VA,NaN,Alexandria,CDBG,NaN,1027042.0
130,2019,VA,NaN,Arlington County,CDBG,NaN,1345258.0
131,2019,VA,NaN,Blacksburg,CDBG,NaN,516140.0
132,2019,VA,NaN,Bristol,CDBG,NaN,257838.0
133,2019,VA,NaN,Charlottesville,CDBG,NaN,393152.0
134,2019,VA,NaN,Chesapeake,CDBG,NaN,1093950.0
135,2019,VA,NaN,Chesterfield County,CDBG,NaN,1410339.0
136,2019,VA,NaN,Christiansburg,CDBG,NaN,98420.0
137,2019,VA,NaN,Colonial Heights,CDBG,NaN,104899.0
138,2019,VA,NaN,Danville,CDBG,NaN,864447.0


In [13]:
#https://data.census.gov/cedsci/table?q=virginia%20poverty%202019%20by%20county&tid=ACSST1Y2019.S2201
pov = pd.read_csv('poverty2019.csv', header=0, nrows=1, usecols =lambda x: x not in ['Label (Grouping)'])
pov

,"Autauga County, Alabama!!Total!!Estimate","Autauga County, Alabama!!Below poverty level!!Estimate","Autauga County, Alabama!!Percent below poverty level!!Estimate","Baldwin County, Alabama!!Total!!Estimate","Baldwin County, Alabama!!Below poverty level!!Estimate","Baldwin County, Alabama!!Percent below poverty level!!Estimate","Barbour County, Alabama!!Total!!Estimate","Barbour County, Alabama!!Below poverty level!!Estimate","Barbour County, Alabama!!Percent below poverty level!!Estimate","Bibb County, Alabama!!Total!!Estimate",...,"Villalba Municipio, Puerto Rico!!Percent below poverty level!!Estimate","Yabucoa Municipio, Puerto Rico!!Total!!Estimate","Yabucoa Municipio, Puerto Rico!!Below poverty level!!Estimate","Yabucoa Municipio, Puerto Rico!!Percent below poverty level!!Estimate","Yauco Municipio, Puerto Rico!!Total!!Estimate","Yauco Municipio, Puerto Rico!!Below poverty level!!Estimate","Yauco Municipio, Puerto Rico!!Percent below poverty level!!Estimate",Virginia!!Total!!Estimate,Virginia!!Below poverty level!!Estimate,Virginia!!Percent below poverty level!!Estimate
0,"54,922","8,340",15.20%,"209,618","21,704",10.40%,"22,417","6,875",30.70%,"20,632",...,47.80%,"33,446","17,766",53.10%,"35,228","17,273",49.00%,"8,201,608","865,691",10.60%


In [14]:
pov.columns = pov.columns.str.replace("!!", "_")

In [15]:
pct_below = [x for x in pov.columns if 'Virginia_Percent' in x and 'West Virginia_Percent' not in x]
pct_df = pov[pct_below].T.reset_index().rename({0:'pct_below_poverty','index':'County_Name'},axis=1)
pct_df.County_Name = pct_df.County_Name.str.split(',').str[0]

In [16]:
below = [x for x in pov.columns if 'Virginia_Below' in x and 'West Virginia_Below' not in x]
below_df = pov[below].T.reset_index().rename({0:'pop_below_poverty','index':'County_Name'},axis=1)
below_df.County_Name = below_df.County_Name.str.split(',').str[0]

In [17]:
pop = [x for x in pov.columns if 'Virginia_Total' in x and 'West Virginia_Total' not in x]
pop_df = pov[pop].T.reset_index().rename({0:'population','index':'County_Name'},axis=1)
pop_df.County_Name = pop_df.County_Name.str.split(',').str[0]

In [18]:
pov_clean = pd.concat([pop_df, below_df, pct_df],axis=1)
pov_clean = pov_clean.loc[:,~pov_clean.columns.duplicated()].copy()

In [19]:
df = pd.merge(pov_clean, susb_va, how='inner', indicator ='matched')

In [20]:
df

,County_Name,population,pop_below_poverty,pct_below_poverty,Employment_Accommodation,Employment_Accommodation_and_Food_Services,Employment_Administrative_and_Support_Services,Employment_Administrative_and_Support_and_Waste_Management_and_Remediation_Services,"Employment_Agriculture,_Forestry,_Fishing_and_Hunting",Employment_Air_Transportation,...,Employment_Transportation_and_Warehousing,Employment_Truck_Transportation,Employment_Utilities,Employment_Warehousing_and_Storage,Employment_Waste_Management_and_Remediation_Services,Employment_Water_Transportation,Employment_Wholesale_Electronic_Markets_and_Agents_and_Brokers,Employment_Wholesale_Trade,Employment_Wood_Product_Manufacturing,matched
0,Accomack County,"32,325","6,141",19.00%,259.0,1010.0,398.0,421.0,20.0,NaN,...,114.0,57.0,168.0,NaN,NaN,NaN,NaN,130.0,NaN,both
1,Albemarle County,"100,293","9,142",9.10%,679.0,4921.0,2298.0,2374.0,109.0,138.0,...,682.0,65.0,NaN,9.0,76.0,NaN,24.0,447.0,154.0,both
2,Alleghany County,"14,830","2,925",19.70%,77.0,400.0,72.0,72.0,NaN,NaN,...,18.0,15.0,NaN,NaN,NaN,NaN,NaN,27.0,96.0,both
3,Amelia County,"12,861","1,375",10.70%,NaN,120.0,61.0,95.0,39.0,NaN,...,104.0,41.0,NaN,NaN,NaN,NaN,3.0,117.0,117.0,both
4,Amherst County,"30,994","4,327",14.00%,15.0,634.0,127.0,135.0,30.0,NaN,...,282.0,228.0,NaN,NaN,NaN,NaN,NaN,175.0,48.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,Suffolk city,"89,418","9,267",10.40%,294.0,3526.0,1189.0,1221.0,7.0,NaN,...,2375.0,258.0,NaN,1982.0,32.0,NaN,16.0,1207.0,NaN,both
129,Virginia Beach city,"439,531","32,183",7.30%,3605.0,25676.0,11269.0,11581.0,24.0,NaN,...,1943.0,656.0,614.0,84.0,312.0,NaN,253.0,3857.0,41.0,both
130,Waynesboro city,"21,830","3,659",16.80%,172.0,1737.0,300.0,300.0,NaN,NaN,...,305.0,168.0,NaN,NaN,NaN,NaN,NaN,227.0,NaN,both
131,Williamsburg city,"10,552","2,188",20.70%,1136.0,4025.0,360.0,366.0,NaN,NaN,...,13.0,NaN,NaN,NaN,NaN,NaN,NaN,171.0,NaN,both


In [21]:
df.query("matched!='both'")

,County_Name,population,pop_below_poverty,pct_below_poverty,Employment_Accommodation,Employment_Accommodation_and_Food_Services,Employment_Administrative_and_Support_Services,Employment_Administrative_and_Support_and_Waste_Management_and_Remediation_Services,"Employment_Agriculture,_Forestry,_Fishing_and_Hunting",Employment_Air_Transportation,...,Employment_Transportation_and_Warehousing,Employment_Truck_Transportation,Employment_Utilities,Employment_Warehousing_and_Storage,Employment_Waste_Management_and_Remediation_Services,Employment_Water_Transportation,Employment_Wholesale_Electronic_Markets_and_Agents_and_Brokers,Employment_Wholesale_Trade,Employment_Wood_Product_Manufacturing,matched


In [22]:
df = df.drop('matched',axis=1)

In [23]:
econ = pd.read_csv('va_economic_indicators.csv')

In [24]:
econ = econ.rename({'GeoName':'County_Name'},axis=1)
econ2019= econ.loc[econ['year'] == 2019]

In [25]:
econ2019.County_Name = econ.County_Name.str.split(',').str[0]
econ2019

C:\Users\brigg\AppData\Local\Temp\ipykernel_7248\4064411413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  econ2019.County_Name = econ.County_Name.str.split(',').str[0]


,Unnamed: 0,County_Name,Earnings by place of work,Employee and self-employed contributions for government social insurance,Employer contributions for employee pension and insurance funds 8,Employer contributions for government social insurance,Equals: Net earnings by place of residence,Farm income 2,Farm proprietors' income,Less: Contributions for government social insurance 5,...,"Plus: Dividends, interest, and rent 7",Plus: Personal current transfer receipts,Population (persons) 3,Proprietors employment,Proprietors' income 9,Supplements to wages and salaries,Total employment,Wage and salary employment,Wages and salaries,year
5300,0,Accomack,1119986,67460,181644,129706,858335,25567,20126,132313,...,355088,392966,32325,3801,77315,246497,19926,16125,796174,2019
5301,1,Albemarle + Charlottesville,8516598,503727,1099244,847618,6196611,-1492,-11201,927536,...,4541613,1185341,157126,27056,850751,1523053,136653,109597,6142794,2019
5302,2,Alexandria (Independent City),10464576,610359,1205799,1148492,10129189,0,0,1184605,...,3239706,927105,159050,26977,794110,1780045,124505,97528,7890421,2019
5303,3,Alleghany + Covington,501496,38180,65423,58456,412284,-444,-783,67408,...,131915,276916,20520,1424,17669,94651,9760,8336,389176,2019
5304,4,Amelia,170244,13804,19010,15622,353626,12812,9223,21615,...,78275,140891,13058,1813,34205,26821,4615,2802,109218,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5401,101,Washington + Bristol,1670965,124175,221479,176628,1428899,-703,-3802,212489,...,588026,830196,70888,8106,152990,309793,39157,31051,1208182,2019
5402,102,Westmoreland,208451,18313,26153,18944,412151,23645,19017,27785,...,193234,207575,18029,2015,43838,35625,5906,3891,128988,2019
5403,103,Wise + Norton,839644,65536,129250,91456,631798,-239,-338,111264,...,175647,561252,41410,2681,43386,174978,19026,16345,621280,2019
5404,104,Wythe,612425,46401,86564,66536,533289,-4515,-6986,79669,...,150210,336554,28618,3180,39163,119832,15203,12023,453430,2019


In [26]:
econ2019.County_Name = econ2019.County_Name.replace(counties)
econreplace = {
    'Albemarle + Charlottesville':'Albemarle County',
    'Alexandria (Independent City)':'Alexandria city',
    'Alleghany + Covington':'Alleghany County',
    'Campbell + Lynchburg':'Campbell County',
    'Carroll + Galax':'Carroll County',
    'Chesapeake (Independent City)':'Chesapeake city',
    'Frederick + Winchester':'Frederick County',
    'Greensville + Emporia':'Greensville County',
    'Hampton (Independent City)':'Hampton city',
    'Henry + Martinsville':'Henry County',
    'James City + Williamsburg':'James City County',
    'Montgomery + Radford':'Montgomery County',
    'Newport News (Independent City)':'Newport News city',
    'Norfolk (Independent City)':'Norfolk city',
    'Pittsylvania + Danville':'Pittsylvania County',
    'Portsmouth (Independent City)':'Portsmouth city',
    'Prince George + Hopewell':'Prince George County',
    'Richmond (Independent City)':'Richmond city',
    'Roanoke (Independent City)':'Roanoke city',
    'Roanoke + Salem':'Roanoke County',
    'Rockingham + Harrisonburg':'Rockingham County',
    'Southampton + Franklin':'Southampton County',
    'Spotsylvania + Fredericksburg':'Spotsylvania County',
    'Suffolk (Independent City)':'Suffolk city',
    'Virginia Beach (Independent City)':'Virginia Beach city',
    'Washington + Bristol':'Washington County',
    'Wise + Norton':'Wise County',
    'York + Poquoson':'York County'
}
econ2019.County_Name = econ2019.County_Name.replace(econreplace)

C:\Users\brigg\AppData\Local\Temp\ipykernel_7248\547231733.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  econ2019.County_Name = econ2019.County_Name.replace(counties)
C:\Users\brigg\AppData\Local\Temp\ipykernel_7248\547231733.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  econ2019.County_Name = econ2019.County_Name.replace(econreplace)


In [27]:
econ2019

,Unnamed: 0,County_Name,Earnings by place of work,Employee and self-employed contributions for government social insurance,Employer contributions for employee pension and insurance funds 8,Employer contributions for government social insurance,Equals: Net earnings by place of residence,Farm income 2,Farm proprietors' income,Less: Contributions for government social insurance 5,...,"Plus: Dividends, interest, and rent 7",Plus: Personal current transfer receipts,Population (persons) 3,Proprietors employment,Proprietors' income 9,Supplements to wages and salaries,Total employment,Wage and salary employment,Wages and salaries,year
5300,0,Accomack County,1119986,67460,181644,129706,858335,25567,20126,132313,...,355088,392966,32325,3801,77315,246497,19926,16125,796174,2019
5301,1,Albemarle County,8516598,503727,1099244,847618,6196611,-1492,-11201,927536,...,4541613,1185341,157126,27056,850751,1523053,136653,109597,6142794,2019
5302,2,Alexandria city,10464576,610359,1205799,1148492,10129189,0,0,1184605,...,3239706,927105,159050,26977,794110,1780045,124505,97528,7890421,2019
5303,3,Alleghany County,501496,38180,65423,58456,412284,-444,-783,67408,...,131915,276916,20520,1424,17669,94651,9760,8336,389176,2019
5304,4,Amelia County,170244,13804,19010,15622,353626,12812,9223,21615,...,78275,140891,13058,1813,34205,26821,4615,2802,109218,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5401,101,Washington County,1670965,124175,221479,176628,1428899,-703,-3802,212489,...,588026,830196,70888,8106,152990,309793,39157,31051,1208182,2019
5402,102,Westmoreland County,208451,18313,26153,18944,412151,23645,19017,27785,...,193234,207575,18029,2015,43838,35625,5906,3891,128988,2019
5403,103,Wise County,839644,65536,129250,91456,631798,-239,-338,111264,...,175647,561252,41410,2681,43386,174978,19026,16345,621280,2019
5404,104,Wythe County,612425,46401,86564,66536,533289,-4515,-6986,79669,...,150210,336554,28618,3180,39163,119832,15203,12023,453430,2019


In [30]:
df = pd.merge(econ2019, susb_va, how='inner')
df2 = pd.merge(pov_clean, df, how='inner')

In [33]:
df2 = df2.drop('Unnamed: 0',axis=1)

In [34]:
df2.to_csv('final.csv')